In [1]:
import pandas as pd
import numpy as np
from gensim import corpora
from gensim.models import VocabTransform

from common import *
import os
import glob
from joblib import Parallel, delayed
import itertools
import multiprocessing
import copy 

cpu_count = multiprocessing.cpu_count() -1
DATA_FOLDER = '../data/'

In [2]:
# all_docs = glob.iglob(os.path.join(DATA_FOLDER, 'FIPS') + '/**/*.txt', recursive=True)
# corpus_file = open(DATA_FOLDER + 'corpus.txt', 'w')
# for item in all_docs:
#     corpus_file.write("%s\n" % os.path.relpath(item, DATA_FOLDER))

In [3]:
# from nltk.tokenize.toktok import ToktokTokenizer 
# toktok = ToktokTokenizer()
# toktok.tokenize('устройство по п . 3 , отличать тем , что оно снабжать устанавливать в головка винт , в который размещать шарик и ролик .  . ')

In [4]:
# nltk.download()

In [ ]:
%%time
fname = join(DATA_FOLDER, 'all_docs.txt')

if not exists(fname):
    all_docs = glob.glob(join(DATA_FOLDER, 'FIPS') + '/**/*.txt', recursive=True)
    with open(fname, mode='wt', encoding='utf-8') as f:
        f.write('\n'.join(all_docs))
else:
    with open(fname, mode='r', encoding='utf-8') as f:
        all_docs = f.read().splitlines()

# dfolder = os.path.join(DATA_FOLDER, 'FIPS/Inventions patents_txt_output/0c/0c')
# all_docs = glob.glob(dfolder + '/**/*.txt', recursive=True)

CPU times: user 31.3 s, sys: 10.3 s, total: 41.6 s
Wall time: 2min 22s


In [ ]:
%%time
parallelizer = Parallel(n_jobs=cpu_count)

# this iterator returns the functions to execute for each task
tasks_iterator = ( delayed(save_corpus)(list_block, os.path.join(DATA_FOLDER, 'BoW'), i) for 
                  i, list_block in enumerate(grouper(len(all_docs)//100, all_docs)) ) 
result = parallelizer( tasks_iterator )

 69%|██████▉   | 8270/11946 [11:23<09:43,  6.30it/s]  

In [ ]:
corps = glob.glob(join(DATA_FOLDER, 'BoW/*corpus.mm'))
dicts = glob.glob(join(DATA_FOLDER, 'BoW/*.dict'))
corps.sort(key=natural_keys)
dicts.sort(key=natural_keys)
pairs = list(zip(dicts, corps))
result = pairs

In [ ]:
%%time
merged_corpus = None
for dic_name, corp_name in result:
    cur_dict = corpora.Dictionary.load(dic_name)
    cur_corp = corpora.MmCorpus(corp_name)
    
    print(dic_name)
    if not merged_corpus:
        dict1 = cur_dict
        merged_corpus = cur_corp
        continue

    cur_to_dict1 = dict1.merge_with(cur_dict)
    merged_corpus = itertools.chain(merged_corpus, cur_to_dict1[cur_corp])
    
dict1.save(join(DATA_FOLDER, 'old.dict'))
corpora.MmCorpus.serialize(join(DATA_FOLDER, 'corpus.mm'), merged_corpus)

### Filter out tokens that appear in less than 5 documents (absolute number) or more than 50% documents

In [ ]:
%%time
# filter the dictionary
old_dict = corpora.Dictionary.load(join(DATA_FOLDER, 'old.dict'))
new_dict = copy.deepcopy(old_dict)
new_dict.filter_extremes(no_below=3, keep_n=None)
new_dict.save(join(DATA_FOLDER, 'filtered.dict'))

In [ ]:
%%time
# now transform the corpus
corpus = corpora.MmCorpus(join(DATA_FOLDER, 'corpus.mm'))
old2new = {old_dict.token2id[token]:new_id for new_id, token in new_dict.iteritems()}
vt = VocabTransform(old2new)

In [ ]:
%%time
corpora.MmCorpus.serialize(join(DATA_FOLDER, 'filtered_corpus.mm'), vt[corpus], id2word=new_dict)

In [ ]:
!cp ../data/*corpus* ~/Yandex.Disk
!cp ../data/*.dict ~/Yandex.Disk
!cp ../data/all_docs.txt ~/Yandex.Disk